In [ ]:
import pandas as pd
import matplotlib as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
from datetime import date, timedelta
from IPython.display import display
import seaborn as sns

In [ ]:
header_by_day = ["#","Country","Date","TotalCases","NewCases","TotalDeaths","NewDeaths","TotalRecovered","NewRecovered",
"ActiveCases","Serious,Critical","Tot Cases/1M pop","Deaths/1M pop","TotalTests","Tests/1M pop","Population","Continent"]

header_by_week = ["#", "Country", "Cases in the last 7 days", "Cases in the preceding 7 days",
"Weekly case /%/ change", "Cases in the last 7 days/1M pop", "Deaths in the last 7 days", 
"Deaths in the preceding 7 days", "Weekly Death /%/ change", "Deaths in the last 7 days/1M pop", "Population", "Continent"]

In [ ]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def convert_numeric(num):
    condition = {",":"","+":"","%":""}
    if num == "nan" or num == " " or num == "  " or num == " N/A":
        return 0

    for i, j in condition.items():
        num = num.replace(i, j)
    if RepresentsInt(num) == True:
        return int(num)
    return float(num)

In [ ]:
df_covid_by_day = pd.read_csv('Corona_by_day.tsv',sep='\t')

In [ ]:
for i in range(3,16):
    df_covid_by_day[header_by_day[i]] = df_covid_by_day[header_by_day[i]].apply(lambda x:convert_numeric(str(x)))
    
df_covid_by_day[header_by_day[16]] = df_covid_by_day[header_by_day[16]].apply(lambda x:str(x).replace(" ",""))
df_covid_by_day['Date'] = pd.to_datetime(df_covid_by_day['Date'],format="%d-%m-%Y")

In [ ]:
df_covid_by_week = pd.read_csv('Corona_by_week.tsv',sep='\t')

In [ ]:
for i in range(2,11):
    df_covid_by_week[header_by_week[i]] = df_covid_by_week[header_by_week[i]].apply(lambda x:convert_numeric(str(x)))
    
df_covid_by_week[header_by_week[-1]] = df_covid_by_week[header_by_week[-1]].apply(lambda x:str(x).replace(" ",""))

In [ ]:
df_covid_by_day

In [ ]:
df_covid_by_day.dtypes

In [ ]:
df_covid_by_week

In [ ]:
df_covid_by_week.dtypes

### Phân chia dataset theo ngày thành 6 châu lục

Tuần không cần vì các chỉ số không cho phép ta groupby, nếu groupby sẽ bị sai lệch số liệu.

In [ ]:
today = date.today()
yesterday = (today - timedelta(1)).strftime("%Y-%m-%d")

Nhớ pull về hoặc check trong dataset xem có dữ liệu ngày trước không, nếu không thông báo Lê Minh Trí gấp !!

In [ ]:
df_covid_yesterday = df_covid_by_day[df_covid_by_day['Date'] == yesterday]

In [ ]:
continent_yesterday_df = df_covid_yesterday.groupby("Continent").sum().drop("nan").reset_index()
continent_yesterday_df = continent_yesterday_df.drop(["#","Tot Cases/1M pop","Deaths/1M pop","Tests/1M pop"],axis=1)
continent_yesterday_df['Date'] = yesterday
continent_yesterday_df['Date'] = pd.to_datetime(continent_yesterday_df['Date'],format="%Y-%m-%d")

Dataframe của 6 châu lục theo ngày hôm qua. Áp dụng cho nhiều chart khác nhau.

In [ ]:
continent_yesterday_df

Dataframe dựa theo thời gian cho line chart.

In [ ]:
df_covid_time = df_covid_by_day.groupby("Date").sum().reset_index()
df_covid_time = df_covid_time.drop(["#","Tot Cases/1M pop","Deaths/1M pop","Tests/1M pop"],axis=1)
df_covid_time

### Câu 1,4,5 + World Chart (Toàn)

---

### Câu 2,3,6 (Trí)

### Pie Chart

Câu 2 : 
Pie Chart để so sánh tỷ lệ % giữa 6 châu lục về các thông số 
- Số ca nhiễm MỚI
- Số ca tử vong MỚI
- Số ca hồi phục MỚI
- Số ca trong cộng đồng

Mỗi thông số là một biểu đồ, 100% chính là tổng 6 châu lục từng thông số

In [ ]:
piechart_plots = continent_yesterday_df[['Continent','NewCases','NewDeaths','NewRecovered','ActiveCases']]

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs = specs, 
subplot_titles=['New Cases', 'New Deaths', 'New Recovered', 'Active Cases'])

fig.add_trace(go.Pie(labels=piechart_plots['Continent'], values=piechart_plots['NewCases']), 1, 1)
fig.add_trace(go.Pie(labels=piechart_plots['Continent'], values=piechart_plots['NewDeaths']), 1, 2)

fig.add_trace(go.Pie(labels=piechart_plots['Continent'], values=piechart_plots['NewRecovered']), 2, 1)
fig.add_trace(go.Pie(labels=piechart_plots['Continent'], values=piechart_plots['ActiveCases']), 2, 2)

---

### Line Chart

Linechart để trực quan theo thời gian 3 thông số NEW (cases/deaths/recovered) trên toàn bộ thế giới.

Lúc đầu là một biểu đồ chung của cả 3, sau đó tách ra riêng ra 3 biểu đồ lần lượt để xem hướng đi hay mô hình theo thời gian có giống nhau hay không.

In [ ]:
linechart_plots = df_covid_time[['Date','NewCases','NewDeaths','NewRecovered']]

In [ ]:
linechart_plots

In [ ]:
fig = px.line(linechart_plots, x="Date", y=['NewCases', 'NewDeaths','NewRecovered'], markers = True, title="World Overview") 
fig.show()

Tiếp theo, ta chia từng châu lục theo thời gian với thông số giống với World.

In [ ]:
continents = ["Europe","NorthAmerica","Asia","SouthAmerica","Australia/Oceania","Africa"]
df_covid_by_day_plots = df_covid_by_day[['Date','Continent','NewCases', 'NewDeaths','NewRecovered']]

In [ ]:
for continent in continents:
    continent_df_day = df_covid_by_day_plots[df_covid_by_day_plots['Continent'] == continent]
    
    #continent_df_day.set_index('Date',inplace=True)
    continent_df_day = continent_df_day.groupby(by='Date').sum().reset_index()
    
    title = continent + " Overview"
    fig = px.line(continent_df_day, x="Date", y=['NewCases', 'NewDeaths','NewRecovered'], markers = True, title=title)
    fig.show()

---

### Scatter plot

Scatterplot để trực quan về mối quan hệ của hai thông số, ca nhiễm / ca chết trong 7 ngày gần đây / 1 triệu dân trên toàn thế giới.

In [ ]:
scatterplot_df_week = df_covid_by_week[['Country','Cases in the last 7 days/1M pop','Deaths in the last 7 days/1M pop']]
fig = px.scatter(scatterplot_df_week, x='Cases in the last 7 days/1M pop', y='Deaths in the last 7 days/1M pop',color='Country')
fig.update_layout(height=500, width=1200)
fig.show()

---

### Heatmap

Heatmap được sử dụng để trực quan độ nóng của 6 châu lục về trường dữ liệu số ca nhiễm mới mỗi ngày. Các trường dữ liệu MỚI khác đều có chung mô hình nên không cần phải trực quan.

In [ ]:
heatmapplot_day_df = df_covid_by_day_plots.groupby(by=['Date','Continent']).sum().reset_index()
heatmapplot_day_df = heatmapplot_day_df[heatmapplot_day_df['NewCases'] != 0]
heatmapplot_day_df['Date'] = heatmapplot_day_df['Date'].dt.strftime('%d-%m')
heatmapplot_day_df['Date'] = pd.Categorical(heatmapplot_day_df['Date'],categories=heatmapplot_day_df['Date'].unique(),ordered=True)

In [ ]:
heatmapplot_day_df_NewCases = heatmapplot_day_df.pivot_table(index='Continent',columns='Date',values='NewCases')
ax = sns.heatmap(heatmapplot_day_df_NewCases)
ax.set_title("Số ca nhiễm mới mỗi ngày")